In [1]:
import requests
import lxml.html
import sqlite3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv

In [2]:
def get_urls():
    url_list = []
    
    response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do')
    root = lxml.html.fromstring(response.content)
    for tr in root.xpath('//*[@id="contents"]/table/tbody/tr') :
        a = tr.xpath('td[3]/a')[0]
        url = 'https://world.moleg.go.kr' + a.get('href')
        url_list.append(url)
    
    return url_list

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('  ', ' ', new_string2) # double space 제거
    new_string4 = re.sub('   ', ' ', new_string3) # double space 제거
    return new_string4

In [4]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [5]:
def get_info(url):
    details = []
    
    browser.get(url)
    title = browser.find_element('xpath','//*[@id="contents"]/div[3]').text
    country = browser.find_element('xpath','//*[@id="contents"]/div[2]/div[2]/strong').text
    upload_date = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[1]/span').text
    views = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[2]/span').text
    trs = browser.find_element('xpath','//*[@id="lgslTrend_CTS"]').text
    
    browser.implicitly_wait(4)
    
    words = remove_punc(trs)
    details.append(title)
    details.append(country)
    details.append(upload_date)
    details.append(views)
    details.append(words)
    
    return details

In [6]:
def db_save_as_csv(data):
    data.to_csv("법제동향_page1.csv", index=False, encoding='utf-8')

In [7]:
from selenium.webdriver.chrome.options import Options

options = Options()

#바이너리 위치 설정
#options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정



urls = get_urls()
df_list = []

for url in urls:
    
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "국가": details[1],
        "등록일자": details[2],
        "조회수": details[3],
        "본문내용": details[4]
    }])
    
    df_list.append(df)
    
    browser.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)
db_save_as_csv(ARTICLE_LIST)

browser.close()
browser.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_6916\3293790730.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정
C:\Users\user\AppData\Local\Temp\ipykernel_6916\3293790730.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정


10 건 저장완료..


In [8]:
with open(r"법제동향_page1.csv", 'r', encoding='utf-8') as f:
    
    document = pd.read_csv(f)
    
    display(document)

,제목,국가,등록일자,조회수,본문내용
0,"말레이시아, 개정 고용법 시행 연기",말레이시아,2022.09.05.,240,말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...
1,"마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정",중국 마카오특별행정구,2022.08.01.,786,마카오 법제동향 마카오 원숭이두창을 전염병 강제신고 대상으로 지정 마카오 카지노...
2,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령,World,2022.07.22.,1594,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...
3,"독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의",독일,2022.07.14.,1077,독일 입법동향 독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...
4,"태국, 「2022년 형법전 개정법 (제29권)」 공포",태국,2022.07.12.,1105,태국 입법동향 태국 2022년 형법전 개정법 제29권 공포 태국 정부는 2021...
5,"UAE, 「외국인 입국 및 거주에 관한 연방법 시행규칙」 통과",아랍에미리트,2022.07.05.,1467,아랍에미리트 입법동향 UAE 외국인 입국 및 거주에 관한 연방법 시행규칙 통과 셰...
6,"대만, 「의료사고 예방 및 의료분쟁 처리에 관한 법률」 제정",대만,2022.07.01.,2296,대만 법제동향 대만 의료사고 예방 및 의료분쟁 처리에 관한 법률 제정 사진 출처...
7,"스페인, 소비자 보호 규정 강화",스페인,2022.06.30.,1047,스페인 법제동향 스페인 소비자 보호 규정 강화 스페인에서 지난 5월 28일 소비자...
8,미국 해운법 개정,미국,2022.06.30.,1199,미국 입법 동향 미국 해운법 개정 사진출처 Pixabay 지난 6월 16일 바이든...
9,"베트남, 7월부터 지역별 최저임금 인상",베트남,2022.06.22.,2690,베트남 입법동향 베트남 7월부터 지역별 최저임금 인상 2022년 6월 12일 베...
